# 📊Sistema de Análisis de Ventas con Proceso ETL

Este notebook implementa un **proceso ETL completo** para consolidar datos de clientes, productos y ventas desde múltiples fuentes y cargarlos en MySQL para análisis.

<h4>Diagramas:</h4>

In [1]:
# Imports y Conexión
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Datos de conexión
DB_USER = "root"
DB_PASS = ""
DB_HOST = "localhost"
DB_NAME = "sales_db"

engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")

## 2. Carga de Datos

Cargo los CSV desde la carpeta `data/`:

- `customers.csv` Información de clientes
- `products.csv` Información de productos
- `orders.csv` Información de pedidos/ventas
- `order_details.csv` Detalles de pedidos

In [2]:
# Cargar CSVs
customers = pd.read_csv("../data/customers.csv")
products = pd.read_csv("../data/products.csv")
orders = pd.read_csv("../data/orders.csv")
order_details = pd.read_csv("../data/order_details.csv")

print(customers.head())
print(products.head())
print(orders.head())
print(order_details.head())

   CustomerID FirstName    LastName                          Email  \
0           1      Noah  Mccullough              kcruz@hotmail.com   
1           2   Felicia    Gonzalez            cwright@hotmail.com   
2           3   Carolyn      Powers          lisariley@hotmail.com   
3           4    Audrey       Scott  shawn13@schaefer-roberson.com   
4           5   Anthony     Johnson           whitemary@norton.biz   

                    Phone              City                        Country  
0     +1-216-864-8880x189      Port Brandon                  Faroe Islands  
1         +1-227-124-1355  Lake Matthewberg  Holy See (Vatican City State)  
2        469.562.8467x547   West Trevorside            Trinidad and Tobago  
3  001-923-153-2095x08316        West Jenna   United States Virgin Islands  
4        237-105-4803x090    Jacksonchester              Wallis and Futuna  
   ProductID ProductName     Category   Price  Stock
0          1   Statement       Sports  875.39    117
1          

## 3. Limpieza y Normalización

- Elimino los duplicados, aplicando algo de normalización
- Relleno valores nulos básicos en los registros
- Aseguro que los tipos de datos sean correctos

In [3]:
# Limpiar y Normalizar

# Quitar duplicados
customers.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)
order_details.drop_duplicates(inplace=True)

# Rellenar nulos básicos
customers.fillna({"Email":"", "Phone":""}, inplace=True)
products.fillna({"Category":"Unknown"}, inplace=True)

# Asegurar tipos
customers["CustomerID"] = customers["CustomerID"].astype(int)
products["ProductID"] = products["ProductID"].astype(int)
orders["OrderID"] = orders["OrderID"].astype(int)
orders["CustomerID"] = orders["CustomerID"].astype(int)
orders["OrderDate"] = pd.to_datetime(orders["OrderDate"], errors="coerce")
order_details["OrderID"] = order_details["OrderID"].astype(int)
order_details["ProductID"] = order_details["ProductID"].astype(int)

## 4. Carga Inicial a MySQL

- Limpio las tablas existentes
- Procedo a insertar clientes, productos y pedidos en la base de datos

In [4]:
# Cargar en MySQL
with engine.begin() as conn:
    # Limpiar tablas antes de insertar
    conn.exec_driver_sql("DELETE FROM order_details")
    conn.exec_driver_sql("DELETE FROM orders")
    conn.exec_driver_sql("DELETE FROM products")
    conn.exec_driver_sql("DELETE FROM customers")

customers.to_sql("customers", engine, if_exists="append", index=False)
products.to_sql("products", engine, if_exists="append", index=False)
orders.to_sql("orders", engine, if_exists="append", index=False)

-1

## 5. Manejo de Duplicados en Detalles de Pedido

- Detecto y busco duplicados por `OrderID` y `ProductID`
- Para despues agrupar y sumar `Quantity` y `TotalPrice`

In [5]:
# Manejar duplicados en order_details sumando cantidades y precios
dups = order_details[order_details.duplicated(subset=["OrderID","ProductID"], keep=False)]
print(dups.head())
print("Duplicados encontrados:", len(dups))

order_details = order_details.drop_duplicates(subset=["OrderID","ProductID"])

# Agrupar duplicados sumando Quantity y TotalPrice
order_details = (
    order_details
    .groupby(["OrderID","ProductID"], as_index=False)
    .agg({"Quantity":"sum","TotalPrice":"sum"})
)

      OrderID  ProductID  Quantity  TotalPrice
773       265       1082         1      421.83
776       265       1082         6     2530.98
1876      640        768        10     9478.30
1878      640        768         7     6634.81
2898      978        229         4      168.24
Duplicados encontrados: 68


## 6. Carga Optimizada de Order Details

- Aqui debido a que son mas de 60 mil registros voy a usar una inserción por bloques (`chunksize`) y multi-insert (`method='multi'`) para que el MySQL pueda recibir los datos en paquetes de 10 mil registros evitando que el servidor se atasque y controlando un poco el rendimiento

In [6]:
# Cargar order_details con optimizaciones chunksize y multi-insert
order_details.to_sql(
    "order_details",
    engine,
    if_exists="append",
    index=False,
    chunksize=10000,   # inserta de 10000 en 10000
    method="multi"    # agrupa filas en un solo INSERT
)

-7

## 7. Validación de Datos Cargados

- Estas consultas rápidas van a confirmar la cantidad de registros en cada tabla

In [ ]:
# Validación
with engine.connect() as conn:
    print(pd.read_sql("SELECT COUNT(*) AS total_customers FROM customers", conn))
    print(pd.read_sql("SELECT COUNT(*) AS total_products FROM products", conn))
    print(pd.read_sql("SELECT COUNT(*) AS total_orders FROM orders", conn))
    print(pd.read_sql("SELECT COUNT(*) AS total_order_details FROM order_details", conn))


   total_customers
0             5000
   total_products
0            2000
   total_orders
0         20000
   total_order_details
0                60123
